# ***projekt numerik***

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import constants as cst

## ***Implicit model***

In [2]:
# delta's discretisation

delta_t = 0.5
delta_r = 0.5
n = 4

def t_i(i):
    return i*delta_t

def r_j(j):
    return j*delta_r

#T_i_plus_one = np.zeros((n-1,n-1))


##  Constants and their implementations

blablabla

In [3]:
#Dichte nach Radius

def rho(r):

    dichte;

    if(r>= 0 or r <= 25):
        dichte = cst.WASSER[0]

    if((r>= 25 or r <= 30) or (r>= 55 or r<=60) ):
        dichte =cst.STAHL[0]

    if(r>= 30 or r <= 55):
        dichte = cst.GLASSWOLLE[0]

    return dichte

# Wärmekapazität nach Radius
def c_p(r):

    cap;

    if(r>= 0 or r <= 25):
        cap = cst.WASSER[1]

    if((r>= 25 or r <= 30) or (r>= 55 or r<=60)):
        cap = cst.STAHL[1]

    if(r>= 30 or r <= 55):
        cap = cst.GLASSWOLLE[1]

    return cap


#Wärmeleitungskonstante nach Radius
def Lambda(r):

    lam;

    if(r>= 0 or r <= 25):
        lam = cst.WASSER[2]

    if((r>= 25 or r <= 30) or (r>= 55 or r<=60) ):
        lam = cst.STAHL[2]

    if(r>= 30 or r <= 55):
        lam = cst.GLASSWOLLE[2]

    return lam

## Definition of the function of time and its implementation



In [4]:
# function definition for explicit solution

def T_i_p1(T_i,n):

    t_i_p1 = []

    # add to T_i,j the follow up values, iterating on the value J=1 to J = n-1
    for j in range (1, n - 1):
    # first and second derivatives respective to r
        if()
        T_prime = T_i[j+1]-T_i[j]/(delta_r)
        T_prime_2 = T_i[j+1]-2*T_i[j]+T_i[j-1]/(delta_r**2)

        t_i_p1.append(T_i[j] + delta_t / (r_j(j) * c_p(r_j(j)) * rho(r_j(j)) * (Lambda(j) * (T_prime+r_j(j) * T_prime_2)) ))
    
    return t_i_p1


In [7]:
Gt=20
n = cst.size_rhor/delta_r

#anfangsbedigung (2a) t(0,r) = 0

T_initial = np.zeros(int(n-1)) # potential rounding error that can happen here, be careful with the delta_r division increments. it should always be a multiple of 5,3 and 2.

# randbedigung 1 (2b) T(t,R) = g(t)
T_R1 = np.ones(int(n-1))*Gt

#randbedigung 2 (2c) T'(t,0) = 0